# Deliverable Notebook

In [79]:
import pandas as pd
pd.set_option('display.max_rows', None)

Drug Classes:
- ANTIDIABETICS
- PAIN
- IMMUNOLOGY
- MENTAL HEALTH
- ADHD
- OTHER CNS
- OSTEOPOROSIS
- ERECTILE DYSFUNCTION
- ONCOLOGICS
- ANTICOAGULANTS
- VIRAL HEPATITIS
- IMMUNOSUPPRESSANTS
- HIV ANTIVIRALS
- ANTIBACTERIALS
- LIPID REGULATORS
- ANTIVIRALS, HERPES
- NERVOUS SYSTEM DISORDERS
- ANTI-OBESITY PREPS, EXCL. DIETETICS
- COUGH COLD, INCL FLU ANTIVIRALS
- VACCINES (PURE, COMB, OTHER)
- ANTIHYPERTENSIVES, PLAIN & COMBO
- ANTI-PARASITICS, ANTIMALARIALS, INSECTICIDES
- DERMATOLOGICS
- BPH (BENIGN PROSTATIC HYPERTROPHY)
- CANCER DETOX AG, ANTI-NAUSEANTS
- RESPIRATORY AGENTS
- SPEC IMMUNOGLOBULINS & ANTITOXIC SERA
- MIOTICS+ANTIGLAUCOMA PREPS
- SYST ANTIFUNGALS
- DIAGNOSTIC EQUIP & ACCESSORIES & ALLERGEN TESTS
- THROMBOPOIETIN AGONISTS
- ANTI-ULCERANTS
- SEX HORMONES (ANDROGENS, OESTROGENS, PROGESTOGENS,
- ANTITUBERCULARS
- OTHER CARDIOVASCULARS
- CORTICOSTEROIDS, PLAIN & COMBO
- HORMONAL CONTRACEPTION, SYSTEMIC&TOPICAL
- GENITO-URINARY ANTIBACTERIALS & ANTISEPTICS & ANTI
- URINARY INCONTINENCE
- HYPOTHALAMIC HORMONES
- GI PRODUCTS
- BLOOD COAGULATION
- OPHTHALMOLOGY, GENERAL
- ANTIGOUT PREPS
- HEMATOPOIETIC GROWTH FACTORS
- NASAL PREPS, TOPICAL
- ANTI-INFLAMMATORY AND DIGESTIVE ENZYMES AND MODULA
- ALLERGY, SYSTEMIC & NASAL
- ALL OTHER THERAPEUTICS
- J07D6 TUBERCULOSIS VACCINES
- EAR TREATMENTS&EYE-EAR COMBOS
- ERYTHROPOIETINS
- MULTIPLE SCLEROSIS
- THYROID ANTI-THYROID AND IODINE PREPS
- OTHER HORMONES
- ANTI-ALZHEIMERS
- BISPHOSPHONATES TUMOR-RELATED & BONY METASTASES
- OCULAR ANTINEOVASCULARISATION
- INTERFERONS, EXCL MS MARKET
- HOSPITAL SOLUTIONS
- ALL OTHER UROLOGICAL
- IMAGING
- OTHER ALIMENTARY TRACT AND METABOLISM PRODUCTS
- ANTI-ANAEMICS, IRON AND ALL COMBINATIONS
- ALL OTHER RESPIRATORY
- POLYVAL IMMUNOGLOBLULINS IV&IM
- GROWTH HORMONES
- LABOUR INDUCERS
- VITAMINS & MINERALS
- OTHER WOMEN'S HEALTH
- OTHER HAEMATOLOGICALS
- A05A1 CHOLERETICS+CHOLEKINETIC
- DIETETICS
- RESP ANTIVIRALS EX FLU
- A.C.T.H.

In [96]:
drug_class = 'VACCINES (PURE, COMB, OTHER)'

In [97]:
# Retrieving all data about the drug class
data = pd.read_csv('../Processed_Data/drugs_pct_changes_monthly.csv')
data = data[data['Major Class'] == drug_class]
data.Date = pd.to_datetime(data.Date, format='%Y_%m')

# Retrieving computing predictions
# prophet_1 = pd.read_csv(f'../Predictive Models/Results/Method 1/{drug_class}.csv')
# prophet_1.date = pd.to_datetime(prophet_1.date)
prophet_2 = pd.read_csv(f'../Predictive Models/Results/Method 2/{drug_class}.csv')
prophet_2.date = pd.to_datetime(prophet_2.date)

In [98]:
# Extracting number of unique drug per month
nb_drugs = data.groupby('Date')['NDC'].nunique().to_frame(name='# drugs').reset_index().rename(columns={'Date': 'date'})

# Extracting number of changes per month
nb_price_change = data.groupby('Date')['Changed'].sum().to_frame(name='# price changes').reset_index().rename(columns={'Date': 'date'})

# Helper function
def weigthed_average(data, quantity, weights):
    try:
        return (data[quantity] * data[weights]).sum() / data[weights].sum()
    except ZeroDivisionError:
        return data[quantity]    
    
# Extracting Sales weighted WAC per month
data['Sales'] = data['WAC'] * data['TRx']
sales_wac = data.groupby('Date').apply(weigthed_average, 'WAC', 'Sales').to_frame(name = 'Sales weighted WAC').reset_index().rename(columns={'Date': 'date'})

# Extracting TRx weighted WAC per month
trx_wac = data.groupby('Date').apply(weigthed_average, 'WAC', 'TRx').to_frame(name = 'TRx weighted WAC').reset_index().rename(columns={'Date': 'date'})

# Extracting Prophet 1 predictions
# prophet_1 = prophet_1[['date', 'yhat']].rename(columns={'yhat': 'Method 1 predictions'})

# Extracting Prophet 2 predictions
prophet_2 = prophet_2[['date', 'yhat']].rename(columns={'yhat': 'Sales weighted WAC predictions'})

# Merging all and formatting
deliverable = nb_drugs.merge(nb_price_change, how='outer', on='date')
deliverable = deliverable.merge(sales_wac, how='outer', on='date')
deliverable = deliverable.merge(trx_wac, how='outer', on='date')
deliverable = deliverable.merge(prophet_2, how='outer', on='date')
deliverable['Year'] = deliverable.date.dt.year
deliverable['Month'] = deliverable.date.dt.month
deliverable.drop('date', axis=1, inplace=True)
deliverable['Class'] = drug_class
deliverable = deliverable[['Class', 'Year', 'Month', '# drugs','# price changes', 'TRx weighted WAC', 'Sales weighted WAC', 'Sales weighted WAC predictions']]

In [99]:
deliverable

,Class,Year,Month,# drugs,# price changes,TRx weighted WAC,Sales weighted WAC,Sales weighted WAC predictions
0,"VACCINES (PURE, COMB, OTHER)",2014,8,57.0,0.0,141.759162,160.117249,NaN
1,"VACCINES (PURE, COMB, OTHER)",2014,9,57.0,0.0,150.190400,165.415886,NaN
2,"VACCINES (PURE, COMB, OTHER)",2014,10,57.0,15.0,169.071422,186.941925,NaN
3,"VACCINES (PURE, COMB, OTHER)",2014,11,59.0,3.0,175.899458,199.299316,NaN
4,"VACCINES (PURE, COMB, OTHER)",2014,12,63.0,0.0,179.180652,204.545616,NaN
5,"VACCINES (PURE, COMB, OTHER)",2015,1,64.0,0.0,173.156080,211.714182,242.436301
6,"VACCINES (PURE, COMB, OTHER)",2015,2,64.0,0.0,173.208354,217.472635,241.347793
7,"VACCINES (PURE, COMB, OTHER)",2015,3,64.0,3.0,185.592364,230.672189,238.904002
8,"VACCINES (PURE, COMB, OTHER)",2015,4,64.0,0.0,184.255484,231.106403,227.905698
9,"VACCINES (PURE, COMB, OTHER)",2015,5,64.0,0.0,176.260716,227.767896,217.485834
